# **PD-L1 Inhibitors QSAR-ML Project**
A machine learning model using the ChEMBL bioactivity data.

## **Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [ ]:
! pip install chembl_webresource_client

## **Importing libraries**

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target protein**

In [ ]:
target = new_client.target
target_query = target.search('CHEMBL4523993')
targets = pd.DataFrame.from_dict(target_query)
targets

### **Select and retrieve bioactivity data**

In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target

Retrieving bioactivity data for **Human PD-L1** (CHEMBL4523993)

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)

In [ ]:
df.head(3)

## **Handling missing data**
If any compounds has missing value for the **standard_value** and **canonical_smiles** column then drop it.

In [ ]:
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

In [ ]:
len(df2.canonical_smiles.unique())

In [ ]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr

## **Data pre-processing of the bioactivity data**

### **Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame**

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]
df3

### **Labeling compounds as active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df4 = pd.read_csv('/content/drive/MyDrive/PDL1/PDL1_preprocessed.csv')

In [ ]:
bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 1000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1:
    bioactivity_threshold.append("potent")
  elif float(i) <= 100:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

In [ ]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

## **Exploratory Data Analysis (EDA)**

## **Install conda and rdkit**

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

## **Calculate Lipinski descriptors**
A set of rule-of-thumb for evaluating the **druglikeness** of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile.

The Lipinski's Rule stated the following:
* Molecular weight < 500 Dalton
* Octanol-water partition coefficient (LogP) < 5
* Hydrogen bond donors < 5
* Hydrogen bond acceptors < 10

### **Import libraries**

In [ ]:
!pip install rdkit-pypi


In [ ]:
from rdkit.Chem import Descriptors, Lipinski


### **Calculate descriptors**

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
import numpy as np
import pandas as pd
def lipinski(smiles, verbose=False):
    moldata = []
    for elem in smiles:
        mol = Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData = np.arange(1, 1)
    i = 0
    for mol in moldata:
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
        desc_NumRotatableBonds = Lipinski.NumRotatableBonds(mol)


        # Calculate TPSA
        desc_TPSA = Descriptors.TPSA(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors,
                        desc_NumRotatableBonds,
                        desc_TPSA
                        ])

        if i == 0:
            baseData = row
        else:
            baseData = np.vstack([baseData, row])
        i += 1

    columnNames = ["MW", "LogP", "NumHDonors", "NumHAcceptors", "NumRotatableBonds", "TPSA"]
    descriptors = pd.DataFrame(data=baseData, columns=columnNames)

    return descriptors

In [ ]:
df_lipinski = lipinski(df5.canonical_smiles)
df_lipinski

### **Combine DataFrames**

In [ ]:
df_lipinski

Now, let's combine the 2 DataFrame

In [ ]:
df_combined = pd.concat([df5,df_lipinski], axis=1)

In [ ]:
df_combined

### **Convert IC50 to pIC50**
To allow **IC50** data to be more uniformly distributed, we will convert **IC50** to the negative logarithmic scale which is essentially **-log10(IC50)**.

This custom function pIC50() will accept a DataFrame as input and will:
* Take the IC50 values from the ``standard_value`` column and converts it from nM to M by multiplying the value by 10$^{-9}$
* Take the molar value and apply -log10
* Delete the ``standard_value`` column and create a new ``pIC50`` column

In [ ]:
import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', axis=1)

    return x

Point to note: Values greater than 100,000,000 will be fixed at 100,000,000 otherwise the negative logarithmic value will become negative.

In [ ]:
df_combined.standard_value.describe()

In [ ]:
-np.log10( (10**-9)* 100000000 )

In [ ]:
-np.log10( (10**-9)* 10000000000 )

In [ ]:
def norm_value(input):
    norm = []
    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)
    input['standard_value_norm'] = norm
    x = input.drop('standard_value', axis=1)
    return x

We will first apply the norm_value() function so that the values in the standard_value column is normalized.

In [ ]:
df_norm = norm_value(df_combined)
df_norm

In [ ]:
df_norm.standard_value_norm.describe()

In [ ]:
df_final = pIC50(df_norm)
df_final

In [ ]:
df_final.pIC50.describe()

In [ ]:
label_mapping = {
    'potent': 'group1',
    'active': 'group1',
    'intermediate': 'group2',
    'inactive': 'group2'
}
df_final['Groups'] = df_final['class'].map(label_mapping)
features = df_final.drop(['Groups'], axis=1)
labels = df_final['Groups']
print(features)
print(labels)

## **Chemical Space Analysis (EDA) via Lipinski descriptors**

### **Import library**

In [ ]:
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt

### **Frequency plot of the 2 bioactivity classes**

In [ ]:
plt.figure(figsize=(5.5, 5.5))
order = ['potent','active', 'intermediate', 'inactive']
sns.countplot(x='class', data=df_final, edgecolor='black', order=order)
plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')
plt.savefig('plot_bioactivity_class.pdf')

### **Scatter plot of MW versus LogP**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
hue_order = ['potent','active', 'intermediate', 'inactive']
plt.figure(figsize=(5.5, 5.5))
sns.scatterplot(x='MW', y='LogP', data=df_final, hue='class', size='pIC50', edgecolor='black', alpha=0.5, hue_order=hue_order)
plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.savefig('plot_MW_vs_LogP.pdf')


### **Box plots**

#### **pIC50 value**

In [ ]:
plt.figure(figsize=(5.5, 5.5))
sns.boxplot(x = 'Groups', y = 'pIC50', data = df_final)
plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('pIC50 value', fontsize=14, fontweight='bold')
plt.savefig('plot_ic50.pdf')

**Statistical analysis | Mann-Whitney U Test**

In [ ]:
def mannwhitney(descriptor, verbose=False):
  # https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
  from numpy.random import seed
  from numpy.random import randn
  from scipy.stats import mannwhitneyu
  seed(1)
  selection = [descriptor, 'Groups']
  df = df_final[selection]
  group1 = df[df['Groups'] == 'group1']
  group1 = group1[descriptor]
  selection = [descriptor, 'Groups']
  df = df_final[selection]
  group2 = df[df['Groups'] == 'group2']
  group2 = group2[descriptor]
  stat, p = mannwhitneyu(group1, group2)
  alpha = 0.05
  if p > alpha:
    interpretation = 'Same distribution (fail to reject H0)'
  else:
    interpretation = 'Different distribution (reject H0)'

  results = pd.DataFrame({'Descriptor':descriptor,
                          'Statistics':stat,
                          'p':p,
                          'alpha':alpha,
                          'Interpretation':interpretation}, index=[0])
  filename = 'mannwhitneyu_' + descriptor + '.csv'
  results.to_csv(filename)

  return results

In [ ]:
mannwhitney('pIC50')

#### **MW**

In [ ]:
plt.figure(figsize=(5.5, 5.5))
order =['group1', 'group2']
sns.boxplot(x = 'Groups', y = 'MW', data = df_final, order = order)
plt.xlabel('MW Comparison', fontsize=14, fontweight='bold')
plt.ylabel('MW', fontsize=14, fontweight='bold')
plt.savefig('plot_MW.pdf')

In [ ]:
mannwhitney('MW')

#### **LogP**

In [ ]:
plt.figure(figsize=(5.5, 5.5))
order =['group1', 'group2']
sns.boxplot(x = 'Groups', y = 'LogP', data = df_final, order = order)
plt.xlabel('LogP Comparison', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.savefig('plot_LogP.pdf')

In [ ]:
mannwhitney('LogP')

#### **NumHDonors**

In [ ]:
plt.figure(figsize=(5.5, 5.5))
order =['group1', 'group2']
sns.boxplot(x = 'Groups', y = 'NumHDonors', data = df_final, order = order)
plt.xlabel('nHD Comparison', fontsize=14, fontweight='bold')
plt.ylabel('nHD', fontsize=14, fontweight='bold')
plt.savefig('plot_NumHDonors.pdf')

In [ ]:
mannwhitney('NumHDonors')

#### **NumHAcceptors**

In [ ]:
plt.figure(figsize=(5.5, 5.5))
order =['group1', 'group2']
sns.boxplot(x = 'Groups', y = 'NumHAcceptors', data = df_final, order = order)

plt.xlabel('nHA Comparison', fontsize=14, fontweight='bold')
plt.ylabel('nHA', fontsize=14, fontweight='bold')


plt.savefig('plot_NumHAcceptors.pdf')

In [ ]:
mannwhitney('NumHAcceptors')

#### **TPSA**

In [ ]:
plt.figure(figsize=(5.5, 5.5))
order =['group1', 'group2']
sns.boxplot(x = 'Groups', y = 'TPSA', data = df_final, order = order)
plt.xlabel('TPSA Comparison', fontsize=14, fontweight='bold')
plt.ylabel('TPSA', fontsize=14, fontweight='bold')
plt.savefig('plot_TPSA.pdf')

In [ ]:
mannwhitney('TPSA')

#### **Number of Rotatable Bonds**

In [ ]:
plt.figure(figsize=(5.5, 5.5))
order =['group1', 'group2']
sns.boxplot(x = 'Groups', y = 'NumRotatableBonds', data = df_final, order = order)
plt.xlabel('nRot Comparison', fontsize=14, fontweight='bold')
plt.ylabel('nRot', fontsize=14, fontweight='bold')
plt.savefig('plot_TPSA.pdf')

In [ ]:
mannwhitney('NumRotatableBonds')

## **Descriptor Calculation and Dataset Preparation**

### **Download PaDEL-Descriptor**

In [ ]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh

In [ ]:
! unzip padel.zip

## **Load bioactivity data**

In [ ]:
selection = ['canonical_smiles','molecule_chembl_id']
df3_selection = df_final[selection]
df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [ ]:
! cat molecule.smi | head -5

In [ ]:
! cat molecule.smi | wc -l

## **Calculate fingerprint descriptors**


### **Calculate PaDEL descriptors**

In [ ]:
! cat padel.sh

In [ ]:
! bash padel.sh

In [ ]:
! ls -l

## **Preparing the X and Y Data Matrices**

### **X data matrix**

In [ ]:
! pip install -U imbalanced-learn

In [ ]:
df3_X = pd.read_csv('/content/drive/MyDrive/PDL1/descriptors_output_PDL1.csv')

In [ ]:
df3_X

In [ ]:
df_final

In [ ]:
df6 = df_final['class']

In [ ]:
df6

In [ ]:
df7 = pd.concat([df3_X,df6], axis=1)


In [ ]:
df7

### **Create a mapping dictionary**


---



In [ ]:
mapping = {'potent': 0, 'active': 1, 'intermediate':2, 'inactive': 3}
df7['bioactivity'] = df7['class'].map(mapping)
df7

In [ ]:
y = df_final['bioactivity']

In [ ]:
y

In [ ]:
X = df7.drop(["class", "bioactivity"], axis=1)

In [ ]:
X

In [ ]:
y.value_counts()

In [ ]:
y.value_counts().plot.pie(autopct='%.2f')

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(sampling_strategy="not majority")
X_res, y_res = ros.fit_resample(X, y)

ax = y_res.value_counts().plot.pie(autopct='%.2f')
_ = ax.set_title("Over-sampling")

In [ ]:
X_res

In [ ]:
df_7 = X_res.drop("Name", axis=1)

In [ ]:
y_res

###**Removing highly correlated features**

In [ ]:
import pandas as pd
if not isinstance(df_7, pd.DataFrame):
    df_7 = pd.DataFrame(df_7)
correlation_matrix = df_7.corr()
high_corr_columns = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.90:
            colname = correlation_matrix.columns[i]
            high_corr_columns.add(colname)
df_7_filtered = df_7.drop(columns=high_corr_columns)
print(f"Removed columns: {high_corr_columns}")

### **Let's examine the data dimension**

In [ ]:
df_7_filtered

In [ ]:
df_7_filtered=X

In [ ]:
X

In [ ]:
Y = y_res

In [ ]:
Y

### **Remove low variance features**

In [ ]:
from sklearn.feature_selection import VarianceThreshold
selection = VarianceThreshold(threshold=(.8 * (1 - .8)))
X = selection.fit_transform(X)

## **4. Data split (80/20 ratio)**

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
X_test.shape, Y_test.shape

In [ ]:
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb

In [ ]:
names = ["Nearest_Neighbors", "Linear_SVM", "Polynomial_SVM", "RBF_SVM", "Gaussian_Process",
         "Gradient_Boosting", "Decision_Tree", "Extra_Trees", "Random_Forest", "Neural_Net", "AdaBoost",
         "Naive_Bayes", "LogisticRegression", "lightgbm"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025, random_state=42),
    SVC(kernel="poly", degree=3, C=0.025, random_state=42),
    SVC(kernel="rbf", C=1, gamma=2, random_state=42),
    GaussianProcessClassifier(1.0 * RBF(1.0), random_state=42),
    GradientBoostingClassifier(n_estimators=500, learning_rate=1.0, random_state=42),
    DecisionTreeClassifier(max_depth=5, random_state=42),
    ExtraTreesClassifier(n_estimators=500, min_samples_split=2, random_state=42),
    RandomForestClassifier(max_depth=3, n_estimators=500, criterion='gini', random_state=42),
    MLPClassifier(random_state=42, hidden_layer_sizes=100),
    AdaBoostClassifier(n_estimators=500, random_state=42),
    LogisticRegression(random_state=42),
    lgb.LGBMClassifier(n_estimators=500, random_state=42),
    GaussianNB()]

###**Evaluate each classifier on Train, CV, and Test sets**

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, matthews_corrcoef
from sklearn.model_selection import cross_val_predict
import pandas as pd
results = []
for name, clf in zip(names, classifiers):
    clf.fit(X_train, Y_train)
    Y_train_pred = clf.predict(X_train)
    accuracy_train = accuracy_score(Y_train, Y_train_pred)
    mcc_train = matthews_corrcoef(Y_train, Y_train_pred)
    Y_cv_pred = cross_val_predict(clf, X_train, Y_train, cv=10)
    accuracy_cv = accuracy_score(Y_train, Y_cv_pred)
    mcc_cv = matthews_corrcoef(Y_train, Y_cv_pred)
    Y_test_pred = clf.predict(X_test)
    accuracy_test = accuracy_score(Y_test, Y_test_pred)
    mcc_test = matthews_corrcoef(Y_test, Y_test_pred)
    results.append({
        "Classifier": name,
        "Accuracy_Train": accuracy_train,
        "MCC_Train": mcc_train,
        "Accuracy_CV": accuracy_cv,
        "MCC_CV": mcc_cv,
        "Accuracy_Test": accuracy_test,
        "MCC_Test": mcc_test
    })
results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
!pip install tabulate
from tabulate import tabulate
print(tabulate(results_df, headers='keys', tablefmt='psql'))